In [31]:
import pandas as pd
import plotly
import json

with open('time.log', 'r') as json_file:
    json_list = list(json_file)
data = []
for json_str in json_list:
    data.append(json.loads(json_str))
df = pd.DataFrame.from_dict(data, orient='columns')
df = df[df['gamma'] != 1]
baseline_time = df.loc[df["gamma"] == 0]['elapsed'][0]
df['speedup'] =  baseline_time / df['elapsed']
df.drop(['logfile', 'max_new_tokens', 'elapsed'], axis=1, inplace=True)

In [33]:
from scipy.optimize import minimize 

def get_alpha(row):
    gamma = row.gamma
    E = row.E_tok
    if gamma == 0:
        return 0.0
    def func(alpha):
        if alpha <= 0 or alpha >= 1:
            return 100
        return (E - (1 - alpha ** (gamma + 1)) / (1 - alpha)) ** 2

    return minimize(func, 0.5).x[0]

df['alpha'] = df.apply(get_alpha, axis=1)
df

,t,gamma,model,draft,sampling,E_tok,speedup,alpha
0,1.0,0,t5-3b,unigram,identity,1.000000,1.000000,0.000000
1,1.0,0,t5-3b,t5-small,identity,1.000000,1.000000,0.000000
2,1.0,0,t5-3b,t5-base,identity,1.000000,1.000000,0.000000
3,1.0,0,t5-3b,unigram,argmax,1.000000,1.000000,0.000000
4,1.0,0,t5-3b,t5-small,argmax,1.000000,1.000000,0.000000
5,1.0,0,t5-3b,t5-base,argmax,1.000000,1.000000,0.000000
7,1.0,5,t5-3b,unigram,identity,1.225615,0.972719,0.184115
8,1.0,9,t5-3b,unigram,identity,1.229779,0.982890,0.186846
9,1.0,2,t5-3b,unigram,argmax,1.178266,1.024009,0.154421
10,1.0,5,t5-3b,unigram,argmax,1.209616,0.975384,0.173314


In [34]:
import plotly.express as px

fig = px.line(df, x="gamma", y="speedup", color='draft', 
              title="Speedup by gamma. T5-3b", markers=True, line_dash='sampling')
fig.show()

In [35]:
import plotly.express as px

fig = px.line(df, x="gamma", y="E_tok", color='draft', 
              title="E(#generated) by gamma. T5-3b", markers=True, line_dash='sampling')
fig.show()

In [39]:
import plotly.express as px

fig = px.scatter(df[df['gamma'] != 0], x="speedup", y="alpha", color='draft', 
              title="Speedup by alpha. T5-3b", symbol='sampling')
fig.show()